In [1]:
from openai import OpenAI
import os
import time
import logging
from datetime import datetime

client = OpenAI()

completion = client.chat.completions.create(
  model="gpt-3.5-turbo-0125",
  messages=[
    {"role": "system", "content": "You are a poetic assistant, skilled in explaining complex programming concepts with creative flair."},
    {"role": "user", "content": "Compose a poem that explains the concept of recursion in programming."}
  ]
)

print(completion.choices[0].message)

ChatCompletionMessage(content="In the realm of code, where mysteries unfold,\nThere lies a powerful secret, ancient and bold.\nIt's called recursion, a loop within a loop,\nA magical spell that programmers can stoop.\n\nLike a fractal pattern, endlessly repeating,\nIt's a technique that keeps data greeting.\nA function that calls itself, oh what a sight,\nCreating beauty in the depths of the byte.\n\nIt breaks problems down into smaller bits,\nSolving them one by one, without any wits.\nEach recursive call delves deeper in the fray,\nUntil the base case brings the light of day.\n\nA dance of iteration, complex and divine,\nInfinite loops, a labyrinth in coding spine.\nRecursive whispers in the language of machines,\nUnlocking the door to programming dreams.\n\nSo fear not the recursion, embrace its grace,\nLet it guide your code in a dazzling race.\nFor in its looping magic, solutions reside,\nA programming enchantment, forever by your side.", role='assistant', function_call=None, tool

In [32]:
from openai import OpenAI
client = OpenAI()
  
assistant = client.beta.assistants.create(
  name="Math Tutor",
  instructions="You are a personal math tutor. Write and run code to answer math questions.",
  tools=[{"type": "code_interpreter"}],
  model="gpt-3.5-turbo-0125",
)

In [33]:
thread = client.beta.threads.create()

In [34]:
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="I need to solve the equation `3x + 11 = 14`. Can you help me?"
)

In [35]:
run = client.beta.threads.runs.create(
    thread_id=thread.id,
    assistant_id=assistant.id,
    instructions="Please address the user as James Bond",
)

In [37]:
def wait_for_run_completion(client, thread_id, run_id, sleep_interval=5):
    """

    Waits for a run to complete and prints the elapsed time.:param client: The OpenAI client object.
    :param thread_id: The ID of the thread.
    :param run_id: The ID of the run.
    :param sleep_interval: Time in seconds to wait between checks.
    """
    while True:
        try:
            run = client.beta.threads.runs.retrieve(thread_id=thread.id, run_id=run_id)
            if run.completed_at:
                elapsed_time = run.completed_at - run.created_at
                formatted_elapsed_time = time.strftime(
                    "%H:%M:%S", time.gmtime(elapsed_time)
                )
                print(f"Run completed in {formatted_elapsed_time}")
                logging.info(f"Run completed in {formatted_elapsed_time}")
                # Get messages here once Run is completed!
                messages = client.beta.threads.messages.list(thread_id=thread_id)
                for data in messages.data:
                    print(data.content[0].text.value)
                last_message = messages.data[0]
                response = last_message.content[0].text.value
                print(f"Assistant Response: {response}")
                break
        except Exception as e:
            logging.error(f"An error occurred while retrieving the run: {e}")
            break
        logging.info("Waiting for run to complete...")
        time.sleep(sleep_interval)


# === Run ===
wait_for_run_completion(client=client, thread_id=thread.id, run_id=run.id)

# ==== Steps --- Logs ==
run_steps = client.beta.threads.runs.steps.list(thread_id=thread.id, run_id=run.id)
#print(f"Steps---> {run_steps.data[0]}")

Run completed in 00:00:06
James Bond, the solution to the equation 3x + 11 = 14 is x = 1.
I need to solve the equation `3x + 11 = 14`. Can you help me?
Assistant Response: James Bond, the solution to the equation 3x + 11 = 14 is x = 1.
